In [36]:
import cv2
import numpy as np
import ipywidgets as widgets 
import IPython.display as display
from ipywidgets import Box

In [37]:
cap = cv2.VideoCapture('./example/Car video2.mp4')

width = int(cap.get(3)) # 가로 길이
height = int(cap.get(4)) # 세로 길이

count = 0

back_img = cv2.imread('./background/back_img.png')
back_img = cv2.resize(back_img, dsize=(width,height), interpolation=cv2.INTER_AREA)

widget1 = widgets.Image(layout = widgets.Layout(border="solid"), width="50%") 
widget2 = widgets.Image(layout = widgets.Layout(border="solid"), width="50%")

items = [widget1, widget2]
box = Box(children=items)
display.display(box)
kernel = np.ones((3,3), np.uint8)
kernel2 = np.ones((4,4), np.uint8)

while True:
    try:
        ret, frame = cap.read()
        if not ret:
            break
        sub_img = cv2.absdiff(frame,back_img)
        
        B,G,R = cv2.split(sub_img)
        ret,B = cv2.threshold(B,35,255,cv2.THRESH_BINARY)
        ret,G = cv2.threshold(G,35,255,cv2.THRESH_BINARY)
        ret,R = cv2.threshold(R,35,255,cv2.THRESH_BINARY)
        
        thres_img = cv2.bitwise_or(B,G)
        thres_img = cv2.bitwise_or(R,thres_img)
        thres_img = cv2.dilate(thres_img,kernel,2)
        thres_img = cv2.erode(thres_img,kernel2, 1)
    
        box_round,temp = cv2.findContours(thres_img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        for i, now in enumerate(box_round):
            area = cv2.contourArea(now)
            if area>110:
                x,y,width,height = cv2.boundingRect(now)
                cv2.rectangle(frame,(x,y),(x+width,y+height),(255,0, 0),2)
        
        widget1.value = cv2.imencode(".png", frame)[1].tobytes()
        widget2.value = cv2.imencode(".png", thres_img)[1].tobytes()
        count +=1
    
    except KeyboardInterrupt:
        break
        
if cap.isOpened():
    cap.release()

Box(children=(Image(value=b'', layout="Layout(border='solid')", width='50%'), Image(value=b'', layout="Layout(…